In [55]:
import torch
import argparse
import numpy as np
import torch.utils.data

from torch import nn, optim
from torch.autograd import Variable
from torchvision import datasets, transforms
from torchvision.utils import save_image


class AutoEncoder(nn.Module):
    def __init__(self, inp_size, hid_size, lmbd = 1):
        super(AutoEncoder, self).__init__()

        self.lmbd = lmbd;
        self.encodeOp = torch.nn.Linear(inp_size, hid_size);
        self.decodeOp = torch.nn.Linear(hid_size, inp_size);
        

    def encode(self, x):
        return self.encodeOp(x);

    def decode(self, h):
        return self.decodeOp(h);

    def forward(self, x):
        
        return self.decode(self.encode(x))


    def loss_function(self, recon_x, x):        
        return self.loss_function2(recon_x, x)
        loss_f = torch.nn.MSELoss()
        mse = torch.nn.functional.mse_loss(recon_x, x);
        rglzr = 0;
        for p in self.parameters():
            rglzr += p.data.norm(p = 1);
            
        return mse + self.lmbd * rglzr;
    
    def loss_function2(self, recon_x, x):
        return recon_x.add(1).sub(x.add(1)).var();
        


def train(model, optimizer, train_loader, test_loader):
    for epoch in range(10):
        model.train()
        train_loss, test_loss = 0, 0
        for data, _ in train_loader:
            #print(data.view(-1)[0], data.view(-1)[1], data.view(-1)[2])
            data = Variable(data).view(-1, 784)
            x_rec = model(data)
            loss = model.loss_function(x_rec, data)
 

            optimizer.zero_grad()

            loss.backward()
  
            optimizer.step()
            train_loss += loss.data[0]
        print('=> Epoch: %s Average loss: %.8f' % (epoch, train_loss / len(train_loader.dataset)))

        model.eval()
        for data, _ in test_loader:
            data = Variable(data, volatile=True).view(-1, 784)
            x_rec = model(data)
            
            test_loss += model.loss_function(x_rec, data).data[0]

        test_loss /= len(test_loader.dataset)
        print('=> Test set loss: %.8f' % test_loss)

        n = min(data.size(0), 8)
        comparison = torch.cat([data.view(-1, 1, 28, 28)[:n], x_rec.view(-1, 1, 28, 28)[:n]])
        save_image(comparison.data.cpu(), 'pics/reconstruction_' + str(epoch) + '.png', nrow=n)
    return model


def test_work():
    print('Start test')
    get_loader = lambda train: torch.utils.data.DataLoader(
        datasets.MNIST('../data', train=train, download=True, transform=transforms.ToTensor()),
        batch_size=50, shuffle=True)
    train_loader, test_loader = get_loader(True), get_loader(False)
    
    try:
        model = AutoEncoder(inp_size=784, hid_size=20, lmbd = 0.5)
        optimizer = optim.Adam(model.parameters(), lr=1e-3)
    except Exception:
        assert False, 'Error during model creation'
        return

    try:
        model = train(model, optimizer, train_loader, test_loader)
    except Exception:
        assert False, 'Error during training'
        return

    test_x = Variable(torch.randn(1, 784))    
    rec_x, hid_x = model(test_x), model.encode(test_x)
    submodules = dict(model.named_children())
    layers_with_params = np.unique(['.'.join(n.split('.')[:-1]) for n, _ in model.named_parameters()])
    
    assert (hid_x.dim() == 2) and (hid_x.size(1) == 20),  'Hidden representation size must be equal to 20'
    assert (rec_x.dim() == 2) and (rec_x.size(1) == 784), 'Reconstruction size must be equal to 784'
    assert len(layers_with_params) <= 6, 'The model must contain not more than 6 layers'
    assert np.all(np.concatenate([list(p.shape) for p in model.parameters()]) <= 800), 'All hidden sizes must be less than 800'
    assert np.all([isinstance(submodules[name], nn.Linear) for name in layers_with_params]), 'All layers with parameters must be nn.Linear'
    print('Success!🎉')

test_work()

#model = AutoEncoder(inp_size=2, hid_size=2, lmbd = 3)
#for p in model.parameters():
#    print(p.data)



Start test
=> Epoch: 0 Average loss: 0.00074621
=> Test set loss: 0.00050495
=> Epoch: 1 Average loss: 0.00049294
=> Test set loss: 0.00047602
=> Epoch: 2 Average loss: 0.00048463
=> Test set loss: 0.00047472
=> Epoch: 3 Average loss: 0.00048383
=> Test set loss: 0.00047429
=> Epoch: 4 Average loss: 0.00048335
=> Test set loss: 0.00047339
=> Epoch: 5 Average loss: 0.00048291
=> Test set loss: 0.00047343
=> Epoch: 6 Average loss: 0.00048233
=> Test set loss: 0.00047283
=> Epoch: 7 Average loss: 0.00048214
=> Test set loss: 0.00047280
=> Epoch: 8 Average loss: 0.00048186
=> Test set loss: 0.00047245
=> Epoch: 9 Average loss: 0.00048180
=> Test set loss: 0.00047240
Success!🎉


In [11]:
0.6682  + 0.6063 + 0.6192  + 0.4594

2.3531

In [39]:
x1 = torch.randn(3, 5)
x2 = torch.randn(3, 5)

#mse = torch.nn.functional.mse_loss(x1,x2)
print(x1.sub(x2).var())

3.1798876465813697


In [51]:
torch.FloatTensor([1,2]).add(1)


 2
 3
[torch.FloatTensor of size 2]